# CORD-19 Software Mentions - Access Study

## Relation between citation characteristics and accessibility for analysis (RQ2)

In order to understand research software projects better, e.g., to
determine the qualities and characteristics that make research software projects
successful under a given definition of success, or that help make them sustainable,
or that underlie their collapse, access to their source code repositories is needed.
We wanted to find out which characteristics of software citations enable this access,
and how they relate to adherence to the software citation principles.

Some observable qualities of software mentions and citations can be linked to
adherence to the software citation principles:
a citation to software follows the principle of *Importance*, 
not mentioning a software violates that principle; 
identifying authors in references follows the *Credit and attribution* principle; 
providing access to a persistently archived version of source code in citations or references follows the *Persistence* principle; 
providing access to the source code more generally follows the *Accessibility* principle,
which is the one that this question is most concerned with;
providing version information for software that was used follows the *Specificity* principle.

The results of this part of the study would provide evidence concerning the suitability
of the software citation principles for software accessibility.

## Methodology

The dataset of 80 software mentions was manually annotated by SD to
classify for adherence to the software citation principles, and the
possibility to directly or indirectly access the source code of the
mentioned software.
The table below presents the annotations for
mention features that were actually found in the dataset. The dataset
itself is available as *CSM\_sampled\_mention\_access.csv*.

  | Code |   Description |
  | :--- | :--- |
  | VER | Version information in reference
  | CVER | Version information near mention
  | CRE | Information in reference allowing for personal credit
  | REP | Link to community repository in reference
  | CACC | Link to source code near mention

> Table: Annotations for accessibility
  and adherence to the software citation principles, for which
  respective mention features could be found in our dataset.

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np

Read the access dataset as a pandas dataframe, and print it as a sanity check.

In [ ]:
df = pd.read_csv(r'../data/access_study/CSM_sampled_mention_access.csv', encoding='unicode_escape', engine='python', index_col=False).fillna(0)
print(df)

### Accessibility

Print the totals for each access type annotation.

The annotations are in column *Access*, where
*D* means *direct access is possible from the mention*,
*I* means _**in**direct access is possible from the mention_ and
*N* means *no access is possible from the mention*.

Also convert the raw values to printable column headers.

In [ ]:
access = df['Access']
access = access.str.replace('N', 'No access')
access = access.str.replace('D', 'Direct access')
access = access.str.replace('I', 'Indirect access')
access_totals = access.value_counts()
access_totals

Calculate the percentages for each of the totals.

In [ ]:
access_percent = access.value_counts(normalize=True)
access_percent100 = access_percent.mul(100).round(1).astype(str)
access_df = pd.DataFrame({'No. of mentions': access_totals, '% of mentions': access_percent100})
access_df

Convert this new dataframe to latex and print it.

In [ ]:
print(access_df.to_latex())

### Adherence to principles

Look at the characteristics that one expects to have in adherence to the software citation principles.

We cannot detect the ommission of mentions for software that was used in reported research, but we can detect the existence of
- author names that allow for credit;
- a link or identifier for a persistently archived version of the software that was used;
- a link that allows us to access the source code;
- a version identifier.

The dataset contains annotations for instances where we found this information:

In [ ]:
expanded_annos = {
    'SOF': 'Software reference',
    'CRE': 'Creditable author information in reference',
    'CACC': 'Link for access to source code near mention',
    'CVER': 'Version information near mention',
    'VER': 'Version information in reference',
    'REP': 'Link to community repository in reference',
    'N': 'No information in adherence to principles',
}

# Render as LaTeX
print(pd.DataFrame(expanded_annos, index = ['Description']).transpose().to_latex(column_format='rl'))

Define a function for exploding cell-values that are comma-separated string lists.

In [ ]:
# We need to expand a) comma-separated mention types, and then b) comma-separated features.
def explode_list(str_list_candidate):
    '''Explodes a comma-separated string list into a Python list.'''
    items = str_list_candidate.split(',')
    items = [i.strip() for i in items]
    return items

Now count the occurrence of annotations.

In [ ]:
# Create data structure with clustering information
clusters = {
    'Author information': {'annos': {'CRE'}},
    'Software links': {'annos': {'CACC', 'REP'}},
    'Version information': {'annos': {'VER', 'CVER'}},
    'Software reference': {'annos': {'SOF'}},
    'No information': {'annos': {'N'}}
}
y = ['Author information',
     'Software links',
     'Version information',
     'Software reference',
     'No information']
x = [
    'CRE','CACC, REP', 'VER, CVER', 'SOF', 'N'
]
cldf = pd.DataFrame(x, y)
print(cldf.to_latex())

# Cluster the annotations
for i in df.index:
    for category in clusters:
        raw_val = df['Accessibility and Principledness'][i]
        vals = explode_list(raw_val)
        for val in vals:
            if val in clusters[category]['annos']:
                # Iterate count if count already exists, or create inital count
                if val in clusters[category]:
                    clusters[category][val] += 1
                else:
                    clusters[category][val] = 1

# Print final data structure as sanity check
clusters


Create a new dataframe to hold the clusters, and sum up occurrences for single features in cluster category.

In [ ]:
# NaN values in dataframe should be filled with zeroes, and dataframe should be transposed
df2 = pd.DataFrame(clusters).fillna(0).transpose()
# Drop the unneeded column showing the annotations belonging to a category
df2 = df2.drop(['annos'], axis=1)
# Add a column showing the category total, summing up the counts of the single subcategories
df2.insert(0, 'Total', df2.sum(axis=1, skipna=True).astype(int))
df2

Render the new dataframe as a LaTeX table.

In [ ]:
print(df2.to_latex(index=False, column_format='rccccccc'))

Render this more nicely as a stacked bar plot.

In [ ]:
%matplotlib inline

# Use matplotlib to create this and following plots
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update(
    {
        # Adjust to your LaTex-Engine
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "text.usetex": True,
        "pgf.rcfonts": False,
        "axes.unicode_minus": False,
    }
)
import matplotlib.pyplot as plt

# Sort by category total in decending order
df2.sort_values(by=['Total'], inplace=True, ascending=True)

# Collect columns to render (exclude Total)
plot_cols = [col for col in df2.columns.tolist() if col not in ['Total']]

# For each of the columns, add a column with the % of mentions (100% = 80) in a new dataframe, and display that
perdf = pd.DataFrame(df2[plot_cols]).transpose()
perdf['%'] = perdf.apply(lambda x: x/80).sum(axis=1)*100
perdf['%'] = perdf['%'].round(decimals=2)
print(perdf.to_latex())

# Colourblind/-friendly colours adapted from https://gist.github.com/thriveth/8560036
my_colors = ['#4daf4a', '#f781bf', '#e41a1c', '#984ea3', '#999999', '#377eb8', '#a65628']

# Create the plot
ax = df2[plot_cols].plot(kind='barh', stacked=True, figsize=(9,3), color=my_colors)
plt.tight_layout()

# Add a title and rotate the x-axis labels to be horizontal
plt.title('Distribution of mention characteristics')
plt.xticks(rotation=0, ha='center')
plt.xlabel('No. of mentions in our sample')

# Add labels to bar sections, unless the value is 0
for c in ax.containers:
    print(c.datavalues)
    ax.bar_label(c, labels = ['' if v == 0 else int(v) for v in c.datavalues], label_type='center')
    
# Save the plot
plt.tight_layout()
# Save as pgf
plt.savefig('mentions-by-annotation.pgf')
# Show the plot
plt.show()

Analyse feature distribution over mention types.

In [ ]:
# Create a new dataframe and initialize it with the columns we want
typ_cols = ['cat', 'subcat', 'feature', 'count']
typ_df = pd.DataFrame(columns=typ_cols)

# Re-cluster, but this time also record mention type
# We need a new map from features to mention types, and from single annotations to mention types
for i in df.index:
    # Get single values for feature annotations and mention types
    acc_raw = df['Accessibility and Principledness'][i]
    acc_vals = explode_list(acc_raw)
    typ_raw = df['Mention Type'][i]
    typ_vals = explode_list(typ_raw)
    # Dive into the feature annotations
    for acc in acc_vals:
        for category in clusters:
            # Find the cluster the annotation we look at belongs to
            if acc in clusters[category]['annos']:
                for typ in typ_vals:
                    # Set up filters to find the single row for this combination of 
                    # category, annotation and mention type
                    mf_1 = typ_df['cat'] == category
                    mf_2 = typ_df['subcat'] == acc
                    mf_3 = typ_df['feature'] == typ
                    # Get index of the single filtered row
                    indices = typ_df.index[mf_1 & mf_2 & mf_3].tolist()
                    # There shold either be a single row, or the row doesn't yet exist. not exist
                    # Assert this.
                    assert len(indices) in [0, 1]
                    target_row = typ_df.loc[mf_1 & mf_2 & mf_3]
                    if target_row.empty:
                        # Initialize a new row with a count of 1.
                        typ_df.loc[len(typ_df.index)] = [category, acc, typ, 1]
                    else:
                        # Double-check that there is exactly 1 row found, and up the count by 1
                        assert len(indices) == 1
                        typ_df.at[indices[0], 'count'] += 1

# Sort the dataframe
typ_df.sort_values(by=['cat', 'subcat', 'feature'], inplace=True)
# Create a new dataframe with hierarchical indices for sanity check
multi_df = typ_df.set_index(['cat', 'subcat', 'feature', 'count'])
# Create a new dataframe that we'll use to plot the results
plot_df = typ_df.sort_values(by=['feature', 'cat', 'subcat', 'count'])
# Show the saniy check dataframe
multi_df

In [ ]:
# Prepare a set and a list of mention types
types = set()
for values in df['Mention Type'].unique():
    types.update(explode_list(values))
types_list = list(types)
# Prepare a set and a list of accessibility annotations
annos = set()
for values in df['Accessibility and Principledness'].unique():
    annos.update(explode_list(values))
annos_list = list(annos)

In [ ]:
%matplotlib inline
# Set the colourblind-accessible colours as default colours
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=my_colors) 
# Reindex the plotting dataframe
plot_df = plot_df.reindex(columns=['feature', 'cat', 'subcat', 'count'])

# Prepare a data structure to be filled from the original dataframe
typ_cluster = {
    'Author information': {'CRE': {}},
    'Software links': {'CACC': {}, 'REP': {}},
    'Version information': {'VER': {}, 'CVER': {}},
    'Software reference': {'SOF': {}},
    'No information': {'N': {}}
}

# We'll have more than one cluster, prepare a dict
clusters = {}

# Use the copy package to deep copy the cluster data structure
import copy

for typ in ['PUB', 'PRO', 'URL', 'INS', 'NAM']:
    # Create a deep copy of the data structure to take up values for this mention type
    _cl = copy.deepcopy(typ_cluster)
    # Create a dedicated dataframe for this mention type
    typ_df = plot_df.loc[plot_df['feature'] == typ]
    # Iterate through the data type and count occurrences
    for i, row in typ_df.iterrows():
        _cl[row['cat']][row['subcat']] = row['count']
    # Add the data structure to the clusters dict
    clusters[typ]= _cl

# Prepare 5 vertically stacked subplots for the 5 clusters, which share the x axis
fig, axes = plt.subplots(nrows = 5, sharex = True, figsize=(10,14))
plt.suptitle('Distribution of mention characteristics over mention types', fontsize='x-large', y=.99, x=.57)

# Set up a colour dict, so that the bars for each feature always have the same colour
color_dict = {'CRE': my_colors[0], 'CACC': my_colors[1], 'REP': my_colors[2], 'VER': my_colors[3], 'CVER': my_colors[4], 'N': my_colors[6], 'SOF': my_colors[5]}

for i, cl in enumerate(clusters.items()):
    # Get data for the respective cluster
    typ = cl[0]
    _df = pd.DataFrame(cl[1]).transpose()
    # Create a horizontal bar plot for the data
    _ax = _df.plot(kind='barh', stacked=True, color=color_dict, ax=axes[i])
    _ax.set_title('Mention characteristics in ' + typ, y=.56, pad=-14, fontsize='small')
    plt.xticks(rotation=0, ha='center')
    plt.xlabel('Number of characteristics found for mention types.')

    # Add values to bar sections
    for c in _ax.containers:
        _ax.bar_label(c, labels = ['' if v == 0 else int(v) for v in c.datavalues], label_type='center')

# Remove the legends from all plots to use a single legend instead
for ax in fig.axes:
    ax.get_legend().remove()

# Create elements for the legend
from matplotlib.patches import Patch
legend_elements = []
for anno in annos:
    legend_elements.append(Patch(facecolor=color_dict[anno], label=anno))

# Create the unified legend for the subplots
fig.legend(handles=legend_elements, ncol=7, 
           bbox_to_anchor=(.30, 0.045),
           loc='lower left', 
           fontsize='small')

# Format, save, and show plot
plt.tight_layout()
plt.savefig('features-over-mentions.pgf')
plt.show()

Map characteristics to principles.

In [ ]:
# Map principles to existing annotations
principles = {
    'SOF': 'Importance', 
    'CRE': 'Credit', 
    'uniq': 'Unique identification', # No evidence in dataset
    'pers': 'Persistence', # No evidence in dataset
    'REP': 'Accessibility', 
    'VER': 'Specificity'
}
score_dict = {}
_score_df = copy.deepcopy(df)
# Calculate adherence score
for i in _score_df.index:
    score_dict[i] = {}
    # Explode accessibility annotations
    annos = explode_list(_score_df['Accessibility and Principledness'][i])
    for principle in principles.items():
        if principle[0] in annos:
            score_dict[i][principle[1]] = True
        else:
            score_dict[i][principle[1]] = False
score_df = pd.DataFrame(score_dict).transpose()
print(score_df)
mentions = score_df.columns

# Also print the totals for each column
for (columnName, columnData) in score_df.iteritems():
    print(columnName)
    print(columnData.value_counts())
    print(u'\n')

# Use the transpose to plot principle on y axis
id_matrix = np.array(score_df[mentions].values, dtype=float).T
principles = score_df.index

fig, ax = plt.subplots(figsize=(15,15))
mat = ax.imshow(id_matrix, cmap='GnBu', interpolation='nearest')
plt.yticks(range(id_matrix.shape[0]), mentions)
plt.xticks(range(id_matrix.shape[1]), principles, fontsize=6)
plt.xlabel('Mention ID')
plt.ylabel('Principle')

plt.tight_layout()
plt.savefig('adherence.pgf')
plt.show()